# 🔬 Systematic Review Pipeline

**Complete AI-powered automation for literature search, deduplication, and abstract screening.**

**📖 Complete documentation**: See `README.md` for detailed setup, configuration options, troubleshooting, and best practices.

## Quick Overview
- **🧠 AI Query Generation**: From protocol documents
- **🔍 PubMed Integration**: Automated literature search
- **🧹 Smart Deduplication**: Cross-database duplicate removal  
- **🎯 AI Screening**: Professional abstract evaluation
- **📊 Excel Exports**: Comprehensive reporting

## 📚 Step 1: Import Required Libraries
Initialize all necessary Python libraries for data processing, API calls, and file handling.

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import xml.etree.ElementTree as ET
import re
from datetime import datetime, timedelta
from pathlib import Path
from openai import OpenAI
from docx import Document

# Session organization: separate cache and export directories
SESSION_ID = datetime.now().strftime('%Y%m%d_%H%M%S')
CACHE_DIR = f"cache/session_{SESSION_ID}"
EXPORT_DIR = f"export_results/session_{SESSION_ID}"

# Create both directories
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print("✅ All libraries imported successfully!")
print(f"📅 Session: {SESSION_ID}")
print(f"💾 Cache: {CACHE_DIR}/")
print(f"📊 Export: {EXPORT_DIR}/")

✅ All libraries imported successfully!
📅 Session: 20251016_232310
💾 Cache: cache/session_20251016_232310/
📊 Export: export_results/session_20251016_232310/


## 🤖 Step 2: LLM Configuration & API Setup
Configure AI models for query generation and screening. Verify API key connectivity.

In [2]:
# LLM Configuration
LLM_CONFIG = {
    'model': 'google/gemini-2.5-flash',
    'query_model': 'google/gemini-2.5-pro',  # For protocol analysis
    'api_base': 'https://openrouter.ai/api/v1',
    'temperature': 0.1,
    'max_tokens': 10000
}

def call_llm(prompt, model=None, temperature=None):
    """Call LLM with prompt."""
    # Get API key
    api_key = os.getenv('OPENROUTER_API_KEY')
    if not api_key:
        print("❌ OPENROUTER_API_KEY not found!")
        print("   Set it with: export OPENROUTER_API_KEY='your-key'")
        return None
    
    # Setup parameters
    model = model or LLM_CONFIG['model']
    temperature = temperature or LLM_CONFIG['temperature']
    
    try:
        # Create client and make request
        client = OpenAI(api_key=api_key, base_url=LLM_CONFIG['api_base'])
        
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=LLM_CONFIG['max_tokens'],
            temperature=temperature
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"❌ LLM Error: {e}")
        return None

# Test API key
print(" Checking API configuration...")
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key:
    print(f"✅ API key found: {api_key[:8]}...{api_key[-4:]}")
    print(f"🤖 LLM ready!")
    print(f"   Model: {LLM_CONFIG['model']}")
    print(f"   Query Model: {LLM_CONFIG['query_model']}")
else:
    print("❌ OPENROUTER_API_KEY not set!")
    print("   Set it with: export OPENROUTER_API_KEY='your-key'")

 Checking API configuration...
✅ API key found: sk-or-v1...fb30
🤖 LLM ready!
   Model: google/gemini-2.5-flash
   Query Model: google/gemini-2.5-pro


## ⚙️ Step 3: Search Query Configuration

**AI Mode**: Leave `USER_DEFINED_QUERY` empty + place `protocol.docx` in directory  
**Manual Mode**: Set your PubMed query in `USER_DEFINED_QUERY` below

In [3]:
# Manual Query Configuration
# Set your own PubMed search query here if you don't want to use AI generation

# Option 1: Leave empty to use AI generation
USER_DEFINED_QUERY = ""

# Option 2: Set your own query (examples below)
# USER_DEFINED_QUERY = '("penile cancer"[tiab] OR "penile carcinoma"[tiab]) AND ("lymphadenectomy"[tiab] OR "lymph node dissection"[tiab])'
# USER_DEFINED_QUERY = '("diabetes"[MeSH Terms] OR "diabetes mellitus"[tiab]) AND ("exercise"[MeSH Terms] OR "physical activity"[tiab]) AND ("randomized controlled trial"[pt] OR "clinical trial"[pt])'


In [4]:
# Study configuration for manual queries
MANUAL_STUDY_CONFIG = {
    'study_title': 'Manual Query Systematic Review',
    'research_topic': 'User-defined search',
    'primary_outcome': 'As defined by user query',
    'existing_database_file': 'database.xlsx',
    'protocol_file': 'protocol.docx',
    'output_prefix': 'ai_search_manual'
}

if USER_DEFINED_QUERY.strip():
    print("✅ Manual query detected!")
    print(f"🔍 Query: {USER_DEFINED_QUERY}")
    print("🤖 AI generation will be skipped")
    
    # Set global variables for manual mode
    SEARCH_QUERY = USER_DEFINED_QUERY.strip()
    STUDY_CONFIG = MANUAL_STUDY_CONFIG
    AI_MODE = False
    
else:
    print("📝 No manual query set - AI generation will be used if protocol.docx is available")
    SEARCH_QUERY = None
    STUDY_CONFIG = None
    AI_MODE = True

📝 No manual query set - AI generation will be used if protocol.docx is available


## 🧠 Step 4: AI Query Generation Functions

In [5]:
def extract_text_from_protocol(file_path):
    """Extract text from protocol document (.docx, .txt, or .pdf)."""
    try:
        if file_path.endswith('.docx'):
            doc = Document(file_path)
            return '\n'.join([p.text for p in doc.paragraphs])
        
        elif file_path.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        
        elif file_path.endswith('.pdf'):
            try:
                import PyPDF2
                text = []
                with open(file_path, 'rb') as f:
                    pdf_reader = PyPDF2.PdfReader(f)
                    for page in pdf_reader.pages:
                        text.append(page.extract_text())
                return '\n'.join(text)
            except ImportError:
                print("⚠️  PyPDF2 not installed. Install with: pip install PyPDF2")
                return None
        
        else:
            print(f"⚠️  Unsupported format: {file_path}")
            print(f"   Supported: .docx, .txt, .pdf")
            return None
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return None

def extract_query_from_response(response_text):
    """Extract query from AI response."""
    
    # Look for QUERY_START...QUERY_END
    pattern = r'QUERY_START\s*(.*?)\s*QUERY_END'
    match = re.search(pattern, response_text, re.DOTALL | re.IGNORECASE)
    
    if match:
        query = match.group(1).strip()
        query = re.sub(r'\s+', ' ', query)  # Clean whitespace
        return query
    
    print(f"⚠️  Could not extract query")
    return None

def generate_pubmed_query_from_protocol(protocol_file_path, model=None):
    """Generate PubMed search query from protocol using AI."""
    print(f"🧠 Generating query from: {protocol_file_path}")
    
    protocol_text = extract_text_from_protocol(protocol_file_path)
    if not protocol_text:
        return None
    
    print(f"📄 Protocol loaded: {len(protocol_text)} chars")
    
    prompt = f"""You are an expert systematic review methodologist specializing in PubMed search strategy development.

TASK: Analyze the protocol below and create a comprehensive, sensitive PubMed search query.

SEARCH STRATEGY REQUIREMENTS:
1. Apply PICO framework (Population, Intervention, Comparator, Outcome)
2. For each concept:
   - Include MeSH terms: [MeSH Terms]
   - Include text words: [tiab] (title/abstract)
   - Add synonyms and related terms
   - Use wildcards (*) for word variations (e.g., diagnos* for diagnosis/diagnostic)
3. Combine terms within concepts with OR
4. Combine concepts with AND
5. Balance sensitivity (finding all relevant) with specificity (avoiding irrelevant)

IMPORTANT:
- Do NOT add date restrictions unless specified in protocol
- Do NOT add language filters unless specified
- Include both specific terms and broader concepts
- Consider spelling variations and abbreviations

OUTPUT FORMAT (mandatory):

QUERY_START
[Your complete PubMed query here - must be valid PubMed syntax]
QUERY_END

EXAMPLE:
QUERY_START
("diabetes mellitus"[MeSH Terms] OR "diabetes mellitus, type 2"[MeSH Terms] OR diabetes[tiab] OR diabetic[tiab]) AND ("exercise"[MeSH Terms] OR "physical activity"[tiab] OR "aerobic exercise"[tiab] OR training[tiab]) AND ("glycated hemoglobin"[MeSH Terms] OR "hemoglobin a1c"[tiab] OR hba1c[tiab])
QUERY_END

---

PROTOCOL TO ANALYZE:
{protocol_text}

Generate the optimized PubMed search query now:"""

    response = call_llm(
        prompt=prompt,
        model=LLM_CONFIG['query_model'],
        temperature=0.3
    )
    
    if not response:
        print("❌ Failed to generate query")
        return None

    query = extract_query_from_response(response)
    
    if query:
        print("✅ Query generated!")
        print(f"🔗 {query}")
        return query
    else:
        print("❌ Extraction failed")
        return None

print("✅ AI Query Generation ready!")
print("   📄 extract_text_from_protocol()")
print("   🎯 generate_pubmed_query_from_protocol()")

✅ AI Query Generation ready!
   📄 extract_text_from_protocol()
   🎯 generate_pubmed_query_from_protocol()


## 🔍 Step 5: Query Generation Execution
Determine search strategy based on configuration and generate final PubMed query.

In [6]:
# Query Generation Workflow
print("🔍 Query Generation...")

if not AI_MODE:
    # Manual query mode
    print(f"✅ Using manual query: {SEARCH_QUERY[:100]}...")
    
elif os.path.exists('protocol.docx'):
    # AI mode - check cache first
    query_cache = f"{CACHE_DIR}/generated_query.txt"
    
    if os.path.exists(query_cache):
        with open(query_cache, 'r', encoding='utf-8') as f:
            SEARCH_QUERY = f.read().strip()
        print(f"📂 Loaded from cache: {SEARCH_QUERY[:100]}...")
    else:
        # Generate new query
        SEARCH_QUERY = generate_pubmed_query_from_protocol('protocol.docx')
        if SEARCH_QUERY:
            with open(query_cache, 'w', encoding='utf-8') as f:
                f.write(SEARCH_QUERY)
            print(f"💾 Cached to: {query_cache}")
        else:
            print("❌ Query generation failed - check API key")
            
else:
    print("❌ No manual query and no protocol.docx found")
    print("   Set USER_DEFINED_QUERY or add protocol.docx")
    SEARCH_QUERY = None

if SEARCH_QUERY:
    print("🚀 Ready to search!")

🔍 Query Generation...
📂 Loaded from cache: (("Penile Neoplasms"[MeSH] OR "penile cancer"[tiab] OR "penile carcinoma"[tiab] OR "cancer of the pe...
🚀 Ready to search!


## 💾 Step 6: Caching System Setup
Configure intelligent caching to avoid redundant API calls and speed up repeated searches.

In [ ]:
# Caching configuration
CACHE_CONFIG = {
    'cache_dir': CACHE_DIR,  # Use session cache directory
    'cache_duration_hours': 24,
    'user_email': 'researcher@example.com',
    'max_results': 1000, # Max results to fetch from PubMed
    'api_delay': 0.5
}

os.makedirs(CACHE_CONFIG['cache_dir'], exist_ok=True)

def get_cache_filename(query):
    """Generate cache filename based on query hash."""
    import hashlib
    query_hash = hashlib.md5(query.encode()).hexdigest()[:8]
    return f"{CACHE_CONFIG['cache_dir']}/pubmed_search_{query_hash}.json"

def is_cache_valid(cache_file):
    """Check if cache file exists and is still valid."""
    if not os.path.exists(cache_file):
        return False
    cache_age = datetime.now() - datetime.fromtimestamp(os.path.getmtime(cache_file))
    return cache_age.total_seconds() < CACHE_CONFIG['cache_duration_hours'] * 3600

def save_to_cache(data, cache_file):
    """Save data to cache file."""
    with open(cache_file, 'w', encoding='utf-8') as f:
        json.dump({'timestamp': datetime.now().isoformat(), 'data': data}, f, indent=2)
    print(f"💾 Data cached to: {cache_file}")

def load_from_cache(cache_file):
    """Load data from cache file."""
    with open(cache_file, 'r', encoding='utf-8') as f:
        cache_data = json.load(f)
    cached_time = datetime.fromisoformat(cache_data['timestamp'])
    print(f"📂 Using cached data from: {cached_time.strftime('%Y-%m-%d %H:%M:%S')}")
    return cache_data['data']

print("🔄 Caching system ready!")


🔄 Caching system ready!


## 🌐 Step 7: PubMed Search Functions

Execute the search and retrieve study details from PubMed.

In [9]:
def search_pubmed_ids(query, max_results=10000, email="researcher@example.com"):
    """Search PubMed and return list of PMIDs."""
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': max_results,
        'retmode': 'xml',
        'email': email,
        'tool': 'systematic_review_pipeline'
    }
    
    response = requests.get(search_url, params=params, timeout=30)
    response.raise_for_status()
    
    root = ET.fromstring(response.content)
    return [id_elem.text for id_elem in root.findall('.//Id')]

def fetch_study_details(pmids, email="researcher@example.com", batch_size=200):
    """Fetch detailed information for list of PMIDs in batches."""
    all_studies = []
    
    for i in range(0, len(pmids), batch_size):
        batch_pmids = pmids[i:i + batch_size]
        print(f"   📥 Fetching batch {i//batch_size + 1}/{(len(pmids)-1)//batch_size + 1}")
        
        fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        params = {
            'db': 'pubmed',
            'id': ','.join(batch_pmids),
            'rettype': 'xml',
            'retmode': 'xml',
            'email': email,
            'tool': 'systematic_review_pipeline'
        }
        
        response = requests.get(fetch_url, params=params, timeout=60)
        response.raise_for_status()
        
        root = ET.fromstring(response.content)
        for article in root.findall('.//PubmedArticle'):
            study_data = parse_article(article)
            if study_data:
                all_studies.append(study_data)
        
        time.sleep(CACHE_CONFIG['api_delay'])
    
    return all_studies

def get_text(element, path, default=''):
    """Helper to safely extract text from XML element."""
    elem = element.find(path)
    return elem.text if elem is not None and elem.text else default

def parse_article(article_xml):
    """Parse PubMed article XML into dictionary."""
    # Basic fields
    data = {
        'PMID': get_text(article_xml, './/PMID'),
        'Title': get_text(article_xml, './/ArticleTitle'),
        'Journal/Book': get_text(article_xml, './/Journal/Title') or get_text(article_xml, './/Journal/ISOAbbreviation'),
        'Create Date': datetime.now().strftime('%Y-%m-%d')
    }
    
    # Abstract (può essere strutturato con label)
    abstract_parts = []
    for abs_elem in article_xml.findall('.//AbstractText'):
        if abs_elem.text:
            label = abs_elem.get('Label', '')
            if label and label.upper() not in ['UNLABELLED', 'UNASSIGNED']:
                abstract_parts.append(f"{label}: {abs_elem.text}")
            else:
                abstract_parts.append(abs_elem.text)
    data['Abstract'] = ' '.join(abstract_parts)
    
    # Authors
    authors = []
    for author in article_xml.findall('.//Author'):
        last = get_text(author, 'LastName')
        first = get_text(author, 'ForeName')
        if last:
            authors.append(f"{last}, {first}" if first else last)
    data['Authors'] = '; '.join(authors)
    
    # Publication Year
    year = get_text(article_xml, './/PubDate/Year')
    if not year:
        medline_date = get_text(article_xml, './/PubDate/MedlineDate')
        if medline_date:
            match = re.search(r'\d{4}', medline_date)
            year = match.group() if match else ''
    data['Publication Year'] = year
    
    # DOI e PMCID
    doi_elem = article_xml.find('.//ArticleId[@IdType="doi"]')
    data['DOI'] = doi_elem.text if doi_elem is not None else ''
    
    pmc_elem = article_xml.find('.//ArticleId[@IdType="pmc"]')
    data['PMCID'] = pmc_elem.text if pmc_elem is not None else ''
    
    # Citation
    if data['Authors'] and data['Title'] and data['Journal/Book']:
        data['Citation'] = f"{data['Authors']}. {data['Title']} {data['Journal/Book']}. {data['Publication Year']}."
    else:
        data['Citation'] = ''
    
    return data

print("🔍 PubMed search functions ready!")


🔍 PubMed search functions ready!


## 🚀 Step 8: Execute PubMed Search
Perform the literature search and retrieve studies based on the configured query.

In [10]:
# Execute PubMed search with caching
print("🔍 Starting PubMed search...")
print(f"📝 Query: {SEARCH_QUERY}")
print("=" * 70)

# Check cache first
cache_file = get_cache_filename(SEARCH_QUERY)

if is_cache_valid(cache_file):
    print("✅ Valid cache found! Loading from cache...")
    all_studies = load_from_cache(cache_file)
    studies_df = pd.DataFrame(all_studies)
else:
    print("🌐 No valid cache found. Fetching from PubMed API...")
    
    # Step 1: Search for PMIDs
    print("\n📋 Step 1: Searching for PMIDs...")
    pmids = search_pubmed_ids(
        query=SEARCH_QUERY,
        max_results=CACHE_CONFIG['max_results'],
        email=CACHE_CONFIG['user_email']
    )
    
    print(f"✅ Found {len(pmids)} studies")
    
    if not pmids:
        print("❌ No studies found. Please check your query.")
        studies_df = pd.DataFrame()
    else:
        # Step 2: Fetch detailed information
        print(f"\n📚 Step 2: Fetching detailed information for {len(pmids)} studies...")
        all_studies = fetch_study_details(
            pmids=pmids,
            email=CACHE_CONFIG['user_email'],
            batch_size=200
        )
        
        # Convert to DataFrame
        studies_df = pd.DataFrame(all_studies)
        
        # Save to cache
        save_to_cache(all_studies, cache_file)
        
        print(f"\n✅ Successfully retrieved {len(studies_df)} studies")

# Display results summary
if not studies_df.empty:
    print("\n" + "=" * 70)
    print("📊 RESULTS SUMMARY")
    print("=" * 70)
    print(f"Total studies retrieved: {len(studies_df)}")
    print(f"\nSample of retrieved studies:")
    print(studies_df[['PMID', 'Title', 'Publication Year', 'Journal/Book']].head())
    print("\n✅ PubMed search completed successfully!")
else:
    print("\n⚠️  No studies to display")


🔍 Starting PubMed search...
📝 Query: (("Penile Neoplasms"[MeSH] OR "penile cancer"[tiab] OR "penile carcinoma"[tiab] OR "cancer of the penis"[tiab] OR "penile tumo*r"[tiab] OR "penile malignan*"[tiab])) AND (("Lymph Node Excision"[MeSH] OR lymphadenectom*[tiab] OR "lymph node dissection*"[tiab] OR "lymph node excision"[tiab] OR "nodal dissection*"[tiab] OR "groin dissection*"[tiab] OR "inguinal dissection*"[tiab] OR "pelvic dissection*"[tiab] OR "inguinal lymphadenectomy"[tiab] OR "pelvic lymphadenectomy"[tiab])) AND (("Neoplasm Recurrence, Local"[MeSH] OR "Recurrence"[MeSH] OR recurren*[tiab] OR relapse*[tiab] OR "local recurrence"[tiab] OR "regional recurrence"[tiab] OR "distant recurrence"[tiab])) AND (("Prognosis"[MeSH] OR "Risk Factors"[MeSH] OR "Survival Analysis"[MeSH] OR "Predictive Value of Tests"[MeSH] OR prognos*[tiab] OR predict*[tiab] OR "risk factor*"[tiab] OR "risk stratification"[tiab] OR determinant*[tiab] OR associat*[tiab] OR clinicopathologic*[tiab] OR clinicopathol

## 🧹 Step 9: Simple Deduplication Functions

In [11]:
def load_existing_databases(database_folder="old_database"):
    """Load and clean all database files from folder."""
    if not os.path.exists(database_folder):
        return pd.DataFrame(), []
    
    files = [os.path.join(database_folder, f) for f in os.listdir(database_folder) 
             if f.endswith(('.xlsx', '.csv', '.xls'))]
    
    if not files:
        return pd.DataFrame(), []
    
    print(f"📚 Loading {len(files)} database(s):")
    studies = []
    
    for db_file in files:
        print(f"   📖 {os.path.basename(db_file)}")
        
        if db_file.endswith('.csv'):
            df = pd.read_csv(db_file).dropna(how='all').drop_duplicates()
            df['Source_Database'] = os.path.basename(db_file)
            studies.append(df)
        else:
            for sheet in pd.ExcelFile(db_file).sheet_names:
                df = pd.read_excel(db_file, sheet_name=sheet)
                orig, df = len(df), df.dropna(how='all').drop_duplicates()
                
                if len(df) > 0:
                    df['Source_Database'] = f"{os.path.basename(db_file)}:{sheet}"
                    studies.append(df)
                    removed = orig - len(df)
                    print(f"      📄 {sheet}: {len(df)} studies" + (f" (-{removed})" if removed else ""))
    
    if studies:
        combined = pd.concat(studies, ignore_index=True)
        print(f"✅ Total: {len(combined)} studies")
        return combined, files
    
    return pd.DataFrame(), files

def normalize_text(text):
    """Normalize: lowercase, no punctuation, no extra spaces."""
    if pd.isna(text):
        return ''
    import string
    return ' '.join(str(text).lower().translate(str.maketrans('', '', string.punctuation)).split())

def check_duplicates(new_df, existing_df):
    """Find duplicates by searching PMID/DOI/title in existing database."""
    if existing_df.empty:
        return []
    
    print(f"🔍 Checking {len(new_df)} new vs {len(existing_df)} existing...")
    
    # Build searchable set
    values = set()
    for col in existing_df.columns:
        for v in existing_df[col].dropna().astype(str):
            v = v.strip()
            if v:
                values.add(v)
                if len(v) > 20:  # Likely title
                    norm = normalize_text(v)
                    if norm:
                        values.add(norm)
    
    print(f"   📊 {len(values)} searchable values")
    
    # Find duplicates
    duplicates = set()
    for idx, study in new_df.iterrows():
        for field in ['PMID', 'DOI', 'Title']:
            if pd.notna(study.get(field, '')):
                val = str(study[field]).strip()
                check_val = normalize_text(val) if field == 'Title' else val
                
                if val in values or (check_val and check_val in values):
                    duplicates.add(idx)
                    print(f"   🔄 {field}: {val[:50]}...")
                    break
    
    print(f"📊 {len(duplicates)} duplicates found")
    return list(duplicates)

def deduplicate_studies(new_df, database_folder="old_database"):
    """Load databases and remove duplicates."""
    print("🧹 Deduplication...")
    print("=" * 50)
    
    existing_df, files = load_existing_databases(database_folder)
    
    stats = {'total_new': len(new_df), 'database_files': files}
    
    if existing_df.empty:
        print("✅ No existing databases - all new")
        stats.update({'duplicates_found': 0, 'unique_studies': len(new_df)})
        return new_df, stats
    
    dup_idx = check_duplicates(new_df, existing_df)
    
    if dup_idx:
        result = new_df.drop(dup_idx).reset_index(drop=True)
        print(f"\n✅ Summary: {len(new_df)} retrieved, {len(dup_idx)} removed, {len(result)} unique")
    else:
        result = new_df
        print(f"\n✅ All {len(new_df)} studies are unique!")
    
    stats.update({'duplicates_found': len(dup_idx), 'unique_studies': len(result)})
    return result, stats

print("🧹 Deduplication ready! Simple PMID/DOI/title matching in all cells")


🧹 Deduplication ready! Simple PMID/DOI/title matching in all cells


## ✨ Step 10: Execute Deduplication

**Process**: Compare against databases in `old_database/` folder using PMID, DOI, and title matching

In [12]:
# Execute deduplication
print("🧹 Starting deduplication process...")
print("=" * 50)

# Check if we have studies to deduplicate
if studies_df.empty:
    print("❌ No studies available for deduplication")
    print("   Please run the PubMed search first")
    deduplicated_df = pd.DataFrame()
    dedup_stats = {'total_new': 0, 'duplicates_found': 0, 'unique_studies': 0, 'database_files': []}
else:
    # Perform deduplication
    deduplicated_df, dedup_stats = deduplicate_studies(studies_df, database_folder="old_database")
    
    # Store results globally for later use
    DEDUPLICATION_STATS = dedup_stats
    
    if not deduplicated_df.empty:
        print(f"\n🎉 Deduplication completed successfully!")
        print(f"   📥 Original studies: {dedup_stats['total_new']}")
        print(f"   🔄 Duplicates removed: {dedup_stats['duplicates_found']}")
        print(f"   ✨ Unique studies remaining: {dedup_stats['unique_studies']}")
        print(f"   📊 Deduplication rate: {(dedup_stats['duplicates_found']/dedup_stats['total_new']*100):.1f}%")
        
        # Show sample of deduplicated studies
        print(f"\n📄 Sample deduplicated studies:")
        for i, (_, row) in enumerate(deduplicated_df.head(3).iterrows()):
            title = row.get('Title', 'No title')
            year = row.get('Publication Year', 'N/A')
            print(f"   {i+1}. [{year}] {title[:70]}...")
    else:
        print("⚠️  All studies were duplicates - no unique studies remaining")

print(f"\n💡 Next step: Export results or proceed to AI screening")


🧹 Starting deduplication process...
🧹 Deduplication...
📚 Loading 1 database(s):
   📖 database_radion.xlsx
      📄 Scopus_392: 397 studies (-1)
      📄 PubMed_326: 332 studies (-4)
      📄 Web_Of_Science_275: 281 studies (-3)
      📄 CochraneLibrary_46: 50 studies (-2)
      📄 Suplementary Search_2: 4 studies (-7)
      📄 Fulltextscreening_42: 47 studies (-6)
      📄 Paper_Final for Extraction_16 : 16 studies
✅ Total: 1127 studies
🔍 Checking 5 new vs 1127 existing...
   📊 10224 searchable values
   🔄 PMID: 40671095...
   🔄 PMID: 40669117...
   🔄 PMID: 40379592...
   🔄 PMID: 40079925...
📊 4 duplicates found

✅ Summary: 5 retrieved, 4 removed, 1 unique

🎉 Deduplication completed successfully!
   📥 Original studies: 5
   🔄 Duplicates removed: 4
   ✨ Unique studies remaining: 1
   📊 Deduplication rate: 80.0%

📄 Sample deduplicated studies:
   1. [2025] Comparing the Perioperative and Oncological Outcomes of Open Versus Mi...

💡 Next step: Export results or proceed to AI screening


## 🎯 Step 11: Extraction of Inclusion/Exclusion Criteria

**AI Analysis**: Extract screening criteria from protocol using the most intelligent model

In [13]:
def extract_screening_criteria_from_protocol(protocol_file='protocol.docx', cache_file=None):
    """Extract inclusion and exclusion criteria from protocol using AI."""
    
    # Use session cache directory if not specified
    if cache_file is None:
        cache_file = f"{CACHE_DIR}/screening_criteria.json"
    
    # Check if we have cached criteria
    if os.path.exists(cache_file):
        print(f"✅ Loading cached criteria from {cache_file}")
        with open(cache_file, 'r', encoding='utf-8') as f:
            criteria = json.load(f)
        print(f"   ✓ Inclusion criteria: {len(criteria['inclusion_criteria'])} items")
        print(f"   ✓ Exclusion criteria: {len(criteria['exclusion_criteria'])} items")
        return criteria
    
    # Extract protocol text
    print(f"📄 Reading protocol from {protocol_file}...")
    protocol_text = extract_text_from_protocol(protocol_file)
    
    if not protocol_text:
        print("❌ Could not read protocol file")
        return None
    
    print(f"✅ Protocol loaded: {len(protocol_text)} characters")
    
    # AI prompt for criteria extraction
    prompt = f"""You are an expert systematic review methodologist. 

TASK: Carefully analyze the protocol below and extract ALL inclusion and exclusion criteria for abstract screening.

REQUIREMENTS:
1. Extract EVERY inclusion criterion mentioned in the protocol
2. Extract EVERY exclusion criterion mentioned in the protocol
3. Be specific and clear - each criterion should be actionable for screening
4. Maintain the original intent and specificity from the protocol
5. Include criteria about: study design, population, intervention, outcomes, language, publication type, etc.

OUTPUT FORMAT (mandatory - valid JSON only):
{{
  "inclusion_criteria": [
    "criterion 1",
    "criterion 2",
    ...
  ],
  "exclusion_criteria": [
    "criterion 1",
    "criterion 2",
    ...
  ]
}}

IMPORTANT: 
- Output ONLY valid JSON, nothing else
- Be comprehensive - include all relevant criteria
- Use clear, specific language

---

PROTOCOL:
{protocol_text}

Extract the criteria now (JSON only):"""

    print("🧠 Analyzing protocol with AI (using query_model)...")
    response = call_llm(
        prompt=prompt,
        model=LLM_CONFIG['query_model'],
        temperature=0.1
    )
    
    if not response:
        print("❌ Failed to get AI response")
        return None
    
    # Extract JSON from response
    try:
        # Try to find JSON in response
        json_match = re.search(r'\{[\s\S]*\}', response)
        if json_match:
            criteria = json.loads(json_match.group())
        else:
            criteria = json.loads(response)
        
        # Validate structure
        if 'inclusion_criteria' not in criteria or 'exclusion_criteria' not in criteria:
            print("❌ Invalid criteria structure")
            return None
        
        # Save to cache
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(criteria, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Criteria extracted and saved to {cache_file}")
        print(f"   ✓ Inclusion criteria: {len(criteria['inclusion_criteria'])} items")
        print(f"   ✓ Exclusion criteria: {len(criteria['exclusion_criteria'])} items")
        
        return criteria
        
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse JSON response: {e}")
        print(f"Response was: {response[:500]}...")
        return None

print("✅ Criteria extraction function ready!")
print("   Function: extract_screening_criteria_from_protocol()")

✅ Criteria extraction function ready!
   Function: extract_screening_criteria_from_protocol()


## 📋 Step 12: Execute Criteria Extraction

Extract and cache screening criteria from the protocol document.

In [14]:
# Extract screening criteria
SCREENING_CRITERIA = extract_screening_criteria_from_protocol()

if SCREENING_CRITERIA:
    print(f"\n📋 INCLUSION ({len(SCREENING_CRITERIA['inclusion_criteria'])} criteria):")
    for i, c in enumerate(SCREENING_CRITERIA['inclusion_criteria'], 1):
        print(f"  {i}. {c}")
    
    print(f"\n🚫 EXCLUSION ({len(SCREENING_CRITERIA['exclusion_criteria'])} criteria):")
    for i, c in enumerate(SCREENING_CRITERIA['exclusion_criteria'], 1):
        print(f"  {i}. {c}")
    
    print("\n✅ Ready for screening!")
else:
    print("❌ Failed - check protocol.docx and API key")

✅ Loading cached criteria from cache/session_20251016_232310/screening_criteria.json
   ✓ Inclusion criteria: 6 items
   ✓ Exclusion criteria: 2 items

📋 INCLUSION (6 criteria):
  1. Study population consists of patients diagnosed with penile cancer who have undergone lymphadenectomy.
  2. Study assesses one or more clinicopathological factors (e.g., lymphovascular invasion, extranodal extension, tumor grade, perineural invasion, nodal involvement) in relation to recurrence.
  3. Study reports on recurrence outcomes, including local, regional, or distant recurrence, or time to recurrence.
  4. Study design is an observational study (cohort, case-control, or cross-sectional) or a clinical trial.
  5. Study is published in a peer-reviewed journal.
  6. Study is published in any language, provided an English translation is available.

🚫 EXCLUSION (2 criteria):
  1. Publication type is a case report, review, editorial, or conference abstract.
  2. Study provides insufficient data on recurr

## 🔬 Step 13: AI Abstract Screening Function

Screen studies against inclusion/exclusion criteria using the standard LLM model.

In [15]:
def screen_studies(studies_df, criteria, delay=1.0):
    """Screen all studies using AI against inclusion/exclusion criteria."""
    
    print(f"🔬 Screening {len(studies_df)} studies (~{len(studies_df)*delay/60:.1f} min)")
    
    # Build criteria lists once
    inc_list = '\n'.join(f"{i+1}. {c}" for i, c in enumerate(criteria['inclusion_criteria']))
    exc_list = '\n'.join(f"{i+1}. {c}" for i, c in enumerate(criteria['exclusion_criteria']))
    
    results = []
    for idx, (_, study) in enumerate(studies_df.iterrows(), 1):
        print(f"[{idx}/{len(studies_df)}] {study.get('Title', 'No title')[:60]}...")
        
        # Build prompt
        prompt = f"""You are a systematic review expert performing abstract screening.

Your task: Evaluate if this study meets the inclusion criteria or should be excluded.

INCLUSION CRITERIA (must meet ALL):
{inc_list}

EXCLUSION CRITERIA (exclude if meets ANY):
{exc_list}

STUDY TO EVALUATE:
Title: {study.get('Title', 'No title')}
Year: {study.get('Publication Year', 'N/A')}
Abstract: {study.get('Abstract', 'No abstract')}

Instructions:
1. Read the abstract carefully
2. Check if the study meets ALL inclusion criteria
3. Check if the study meets ANY exclusion criteria
4. Make your decision: INCLUDE only if it meets all inclusion criteria AND no exclusion criteria

Output format (mandatory):
DECISION: [INCLUDE or EXCLUDE]
REASON: [Brief explanation citing specific criteria]"""

        # Call AI
        response = call_llm(prompt, temperature=0.1)
        
        # Parse response
        if response:
            decision_match = re.search(r'DECISION:\s*(INCLUDE|EXCLUDE)', response, re.IGNORECASE)
            reason_match = re.search(r'REASON:\s*(.+?)(?:\n|$)', response, re.IGNORECASE | re.DOTALL)
            decision = decision_match.group(1).upper() if decision_match else 'UNCLEAR'
            reason = (reason_match.group(1).strip() if reason_match else 'No reason')[:500]
        else:
            decision, reason = 'ERROR', 'No AI response'
        
        # Store result with study data
        results.append({
            'decision': decision,
            'reason': reason,
            'PMID': study.get('PMID', ''),
            'Title': study.get('Title', ''),
            'Authors': study.get('Authors', ''),
            'Publication Year': study.get('Publication Year', ''),
            'Journal/Book': study.get('Journal/Book', ''),
            'Abstract': study.get('Abstract', ''),
            'DOI': study.get('DOI', ''),
            'Citation': study.get('Citation', '')
        })
        
        emoji = "✅" if decision == 'INCLUDE' else "❌" if decision == 'EXCLUDE' else "⚠️"
        print(f"{emoji} {decision}: {reason[:80]}...")
        
        if idx < len(studies_df):
            time.sleep(delay)
    
    return pd.DataFrame(results)

print("✅ Screening function ready!")

✅ Screening function ready!


## 🚀 Step 14: Execute Screening

Screen all deduplicated studies using AI and the extracted criteria.

In [16]:
# Execute screening
print("🔬 Starting AI-powered abstract screening...")
print("=" * 70)

# Verify we have everything we need
if deduplicated_df.empty:
    print("❌ No deduplicated studies available")
    print("   Please run the deduplication step first")
    screening_results_df = pd.DataFrame()
elif not SCREENING_CRITERIA:
    print("❌ No screening criteria available")
    print("   Please run the criteria extraction step first")
    screening_results_df = pd.DataFrame()
else:
    # Perform screening
    screening_results_df = screen_studies(
        studies_df=deduplicated_df,
        criteria=SCREENING_CRITERIA,
        delay=1.0
    )
    
    # Display summary
    if not screening_results_df.empty:
        print("\n" + "=" * 70)
        print("📊 SCREENING SUMMARY")
        print("=" * 70)
        
        total = len(screening_results_df)
        included = len(screening_results_df[screening_results_df['decision'] == 'INCLUDE'])
        excluded = len(screening_results_df[screening_results_df['decision'] == 'EXCLUDE'])
        unclear = len(screening_results_df[screening_results_df['decision'] == 'UNCLEAR'])
        errors = len(screening_results_df[screening_results_df['decision'] == 'ERROR'])
        
        print(f"Total studies screened: {total}")
        print(f"✅ INCLUDED: {included} ({included/total*100:.1f}%)")
        print(f"❌ EXCLUDED: {excluded} ({excluded/total*100:.1f}%)")
        if unclear > 0:
            print(f"⚠️  UNCLEAR: {unclear} ({unclear/total*100:.1f}%)")
        if errors > 0:
            print(f"🔴 ERRORS: {errors} ({errors/total*100:.1f}%)")
        
        # Show sample of included studies
        included_studies = screening_results_df[screening_results_df['decision'] == 'INCLUDE']
        if not included_studies.empty:
            print(f"\n📄 Sample of INCLUDED studies:")
            for i, (_, row) in enumerate(included_studies.head(3).iterrows(), 1):
                print(f"{i}. {row['Title'][:70]}...")
                print(f"   Reason: {row['reason'][:100]}...")
        
        print("\n✅ Screening completed!")
        print("💡 Next step: Export results")
    else:
        print("\n⚠️ No screening results generated")



🔬 Starting AI-powered abstract screening...
🔬 Screening 1 studies (~0.0 min)
[1/1] Comparing the Perioperative and Oncological Outcomes of Open...
❌ EXCLUDE: The study is a systematic review and meta-analysis, which is an exclusion criter...

📊 SCREENING SUMMARY
Total studies screened: 1
✅ INCLUDED: 0 (0.0%)
❌ EXCLUDED: 1 (100.0%)

✅ Screening completed!
💡 Next step: Export results


## 📊 Step 15: Export Results

Save all results to organized session folder with timestamp.


In [17]:
# Export all results to session folder
print("📊 Exporting results...")

# All exports go in the export directory
print(f"📁 Exporting to: {EXPORT_DIR}/")

# 1. Export PubMed search query (TXT)
if SEARCH_QUERY:
    query_file = f"{EXPORT_DIR}/search_query.txt"
    with open(query_file, 'w', encoding='utf-8') as f:
        f.write(f"PubMed Search Query\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"{'=' * 70}\n\n")
        f.write(SEARCH_QUERY)
    print(f"✅ Query saved: {query_file}")

# 2. Export screening criteria (JSON)
if SCREENING_CRITERIA:
    criteria_file = f"{EXPORT_DIR}/screening_criteria.json"
    with open(criteria_file, 'w', encoding='utf-8') as f:
        json.dump(SCREENING_CRITERIA, f, indent=2, ensure_ascii=False)
    print(f"✅ Criteria saved: {criteria_file}")

# 3. Export deduplicated studies (XLSX)
if not deduplicated_df.empty:
    dedup_file = f"{EXPORT_DIR}/deduplicated_studies.xlsx"
    deduplicated_df.to_excel(dedup_file, index=False)
    print(f"✅ Deduplicated studies saved: {dedup_file}")
    print(f"   ({len(deduplicated_df)} studies)")

# 4. Export screening results (XLSX)
if not screening_results_df.empty:
    screening_file = f"{EXPORT_DIR}/screening_results.xlsx"
    screening_results_df.to_excel(screening_file, index=False)
    print(f"✅ Screening results saved: {screening_file}")
    
    included = len(screening_results_df[screening_results_df['decision'] == 'INCLUDE'])
    excluded = len(screening_results_df[screening_results_df['decision'] == 'EXCLUDE'])
    print(f"   ({included} included, {excluded} excluded)")

print(f"\n🎉 Export completed!")
print(f"📊 Results: {EXPORT_DIR}/")
print(f"💾 Cache: {CACHE_DIR}/")


📊 Exporting results...
📁 Exporting to: export_results/session_20251016_232310/
✅ Query saved: export_results/session_20251016_232310/search_query.txt
✅ Criteria saved: export_results/session_20251016_232310/screening_criteria.json
✅ Deduplicated studies saved: export_results/session_20251016_232310/deduplicated_studies.xlsx
   (1 studies)
✅ Screening results saved: export_results/session_20251016_232310/screening_results.xlsx
   (0 included, 1 excluded)

🎉 Export completed!
📊 Results: export_results/session_20251016_232310/
💾 Cache: cache/session_20251016_232310/
